# A/B testing: RCAF function 20221001

В данном notebook отдел качества сети комании российского оператора проводит A/B test, формулируется гипотеза, тестируется, и проводится финальное интепритация результата. Предоставленные данные собранны с коммерческой сети оператора, которые содержат результаты A/B test группы без применения нового функционала и группы с раскатаным функционалом (without_RCAF vs. with_RCAF). 

Что собираемся делать:

1. [Дизайн эксперимента](#1.-Дизайн-эксперимента)
2. [Сбор и подготовка данных](#2.-Сбор-и-подготовка-данных)
3. [Представление результатов](#3.-Представление-результатов)
4. [Тестирование гипотезы](#4.-Тестирование-гипотезы)
5. [Итоги теста](#5.-Итоги-теста)

Описание функционала:

> Данный функционал разработан с целью увеличения скорости абонентов на высокозагруженных сотах в часы пик. Функционал определяет приоритеты абонентов и устанавливает правила ограничения по скорости на сервисы для низкоприоритетных и назначает освобожденные ресурсы для высокоприоритетных абонентов. Для тестирования выбран город в Южном регионе РФ, определены соты и часы с высокой утилизацией. Абонент попадает в группу тестирования только с правилом, что высокоутилизированная сота для него со статусом 'любимая' (самая часто используемая). Средняя скорость для отображения видео сервиса с примелемым качеством равняется 3Mbps. Мы знаем, что текущий коэффициент конверсии составляет около 15% в среднем в данном регионе. Это значит что **15% абонентов на перегруженных сотах имеют скорость более 3Mbps**. Компания ожидает получить профит при увеличении на 3%, это означает, что **новый функционал будет считаться успешным, если он повысит коэффициент конверсии до 18%.**


***
## 1. Дизайн эксперимента

### Формулирование гипотезы

Сформулируем гипотезу в начале нашего проекта. Это позволит убедиться, что наша интерпретация результатов будет правильной.

Поскольку мы не знаем, будет ли новый функционал работать лучше или хуже (или так же?), как наш текущий режим, мы выберем **two-tailed test**:

$$H_0: p = p_0$$
$$H_a: p \ne p_0$$

где $p$ и $p_0$ обозначают коэффициент конверсии функциона и старого режима соответственно. Так же установим **confidence level of 95%**:

$$\alpha = 0.05$$

Значение $\alpha$ — это порог, который мы устанавливаем, при котором мы говорим: «Если вероятность наблюдения результата как экстремального или более (значение $p$) ниже, чем $\alpha$, то мы отвергаем нулевую гипотезу». . Поскольку наш $\alpha=0,05$ (что указывает на вероятность 5%), наша confidence level (1 - $\alpha$) составляет 95%.


### Выбор переменных

Для начала выберем **две группы**:
* A `control` group - исполльзуется старый режим сети
* A `treatment` (наша экспериментная) группа - для которой будет расскатан новый функционал

Это будет наша *Независимая переменная*. Причина, по которой у нас есть две группы, несмотря на то, что мы знаем базовый коэффициент конверсии, заключается в том, что мы хотим контролировать другие переменные, которые могут повлиять на наши результаты, такие как сезонность и аварии на сети: имея «контрольную» группу, мы можем напрямую сравнивать их результаты с группой «тестовой», потому что единственное систематическое различие между группами заключается в новом функционале.

Для нашей *зависимой переменной* (т.е. то, что мы пытаемся измерить) нас интересует "коэффициент конверсии". Мы можем закодировать это для каждого сеанса пользователя с помощью двоичной переменной:
* `0` - абонент со скоростью ниже 3Mbps на перегруженной соте
* `1` - абонент со скоростью выше 3Mbps на перегруженной соте

Таким образом, мы можем легко рассчитать среднее значение для каждой группы, чтобы получить коэффициент конверсии для каждого режима.

### Выбор размера сэмпла

Важно отметить, что поскольку мы не будем тестировать всю абонентскую базу.

Количество людей , которые мы решим включить в каждую группу, повлияет на точность наших оценочных коэффициентов конверсии: **чем больше размер выборки**, **тем выше шанс обнаружить разницу** в двух группах, если она присутствует.

С другой стороны, чем больше становится наша выборка, тем дороже становится наше исследование.

Необходимый нам размер выборки оценивается с помощью анализа мощности, и это зависит от нескольких факторов:
* **Мощность теста** ($1 - \beta$) - это вероятность обнаружения статистической разницы между группами в нашем тесте, когда разница действительно присутствует. Как правило, это значение равно 0,8
* **Альфа-значение** ($\alpha$) — критическое значение, которое мы установили ранее равным 0,05.
* **Величина эффекта** – ожидаемая разница между коэффициентами конверсии.

Поскольку нашу команду устроила бы разница в 3 %, мы можем использовать 15 % и 18 % для расчета ожидаемой величины эффекта.

In [ ]:
# импорт библиотек
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

%matplotlib inline

# plot styling preferences
plt.style.use('seaborn-whitegrid')
font = {'family' : 'Helvetica',
        'weight' : 'bold',
        'size'   : 14}

mpl.rc('font', **font)

In [ ]:
effect_size = sms.proportion_effectsize(0.15, 0.18)    # Расчет размера эффекта 

required_n = sms.NormalIndPower().solve_power(
    effect_size, 
    power=0.8, 
    alpha=0.02, 
    ratio=1
    )                                                  # расчет размера выборки

required_n = ceil(required_n)                          # округление до следующего целого числа                          

print(required_n)

Нам потребуется **не менее 3067 наблюдений для каждой группы**.

Установка параметра «мощность» на 0,8 на практике означает, что если существует фактическая разница в коэффициенте конверсии между нашими дизайнами, при условии, что разница является той, которую мы оценили (15% против 18%), у нас есть около 80% шансов на успех обнаружить его как статистически значимое в нашем тесте с рассчитанным нами размером выборки.

***
## 2. Сбор и подготовка данных

1. Набор данных мы берем CoreNetwork статистики по каждому абоненту
2. Прочитаем данные в pandas DataFrame
3. Проверим и очистим данные по мере необходимости.
4. Минимальный размер выборки n=3067 из DataFrame для каждой группы 

In [ ]:
df = pd.read_csv('ab_data_1.csv', sep = ';')

df.head()

In [ ]:
df.info()

В DataFrame **294478 строк**, каждая из которых представляет сеанс пользователя, а также **5 столбцов**:
* `user_id` - идентификатор пользователя каждой сессии
* `timestamp` - отметка времени сеанса
* `group` — к какой группе был отнесен пользователь для этого сеанса {`control`, `treatment`}
* `mode` — какой дизайн видел каждый пользователь в этом сеансе {`old_page`, `new_page`}
* `speed` - закончился ли сеанс конверсией или нет (двоичный, `0`=ниже 3Mbps, `1`=выше 3Mbps)

Прежде чем мы приступим к выборке данных для получения нашего подмножества, давайте удостоверимся, что нет пользователей, которые были отобраны несколько раз.

In [ ]:
session_counts = df['user_id'].value_counts(ascending=False)
multi_users = session_counts[session_counts > 1].count()

print(f'{multi_users} users которые появляются несколько раз в наборе данных')

На самом деле есть пользователи, которые появляются более одного раза. Поскольку число довольно низкое, мы продолжим и удалим их из DataFrame, чтобы избежать повторной выборки одних и тех же пользователей.

In [ ]:
users_to_drop = session_counts[session_counts > 1].index

df = df[~df['user_id'].isin(users_to_drop)]
print(f'Обновленный набор данных теперь имеет {df.shape[0]} записей')

### Выборка

Теперь, когда наш DataFrame в порядке, можем продолжить и отобрать n=3067 записей для каждой из групп. Для этого мы можем использовать метод DataFrame.sample() от pandas, который выполнит для нас простую случайную выборку.


In [ ]:
control_sample = df[df['group'] == 'control'].sample(n=required_n, random_state=25)
treatment_sample = df[df['group'] == 'treatment'].sample(n=required_n, random_state=25)

ab_test = pd.concat([control_sample, treatment_sample], axis=0)
ab_test.reset_index(drop=True, inplace=True)

In [ ]:
ab_test.info()

In [ ]:
ab_test['group'].value_counts()

In [ ]:
ab_test

***
## 3. Представление результатов

Первое, что мы можем сделать, это рассчитать некоторые **базовые статистические данные**, чтобы получить представление о том, как выглядят наши выборки.

Напомню:
Стандартное отклонение измеряет, насколько разбросаны значения в наборе данных. Стандартная ошибка — это стандартное отклонение среднего значения в повторных выборках из совокупности.

In [ ]:
conversion_rates = ab_test.groupby('group')['speed']

std_p = lambda x: np.std(x)              # Std. deviation Стандартное отклонение
se_p = lambda x: stats.sem(x)            # Std. error of the proportion (std / sqrt(n)) стандартная ошибка среднего

conversion_rates = conversion_rates.agg([np.mean, std_p, se_p])
conversion_rates.columns = ['conversion_rate', 'std_deviation', 'std_error']


conversion_rates.style.format('{:.3f}')

Судя по приведенной выше статистике, похоже, что новый функционал работает лучше, **Коэффициент конверсии 15,3% против 18,7%**.

In [ ]:
plt.figure(figsize=(8,6))

sns.barplot(x=ab_test['group'], y=ab_test['speed'], ci=False)

plt.ylim(0, 0.17)
plt.title('Conversion rate by group', pad=20)
plt.xlabel('Group', labelpad=15)
plt.ylabel('Converted (proportion)', labelpad=15);

Является ли эта разница статистически значимой?

***
## 4. Тестирование гипотезы

Последним шагом нашего анализа является проверка нашей гипотезы. Так как у нас очень большая выборка, мы можем использовать  normal approximation для расчета нашего значения $p$ (т.е. z-тест).

Мы будем использовать модуль `statsmodels.stats.proportion`, чтобы получить значение $p$ и доверительные интервалы:

In [ ]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [ ]:
control_results = ab_test[ab_test['group'] == 'control']['speed']
treatment_results = ab_test[ab_test['group'] == 'treatment']['speed']

In [ ]:
n_con = control_results.count()
n_treat = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'ci 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

***
## 5. Итоги теста

Поскольку наше значение $p$ на много ниже значения $\alpha$=0,05, мы можем отвергнуть нулевую гипотезу $H_0$, а это означает, что наш функционал имеет существенные отличия от нашего старого режима.

Кроме того, если мы посмотрим на доверительный интервал для «тестовой» группы ([0.173, 0.201], то есть 18,5-18,9%), то заметим, что:
1. Он не включает наше базовое значение коэффициента конверсии 15%.
2. Он включает наше целевое значение в 18 % (3 % роста, к которому мы стремились).

Это еще одно доказательство того, что наш новый функционал будет улучшением нашего старого режима работы!